# DecisionCell  
A modular multi-agent framework for transparent, tool-augmented decision making The system combines:

- **ReAct-style reasoning** – agents think, take structured actions, call tools, and react to observations  
- **Transparent reasoning traces** – each agent exposes its step-by-step thought process  
- **Retrieval-Augmented Generation (RAG)** – agents can query their own knowledge bases to ground reasoning in data  

This notebook walks you through the full setup required to run a DecisionCell experiment.

In [1]:
from core.Agent import Agent
from core.Logbook import Logbook
from core.Tools import ask_human, agent_as_tool, kb_search, calculate

# Load configs 
import yaml
from pathlib import Path

ModuleNotFoundError: No module named 'langchain'

## 1. Set up the Staff (agents)

DecisionCell works with a small set of collaborating agents.  
Each agent is defined by:

- A **YAML config** (name, role, provider, model, system prompt)
- Optional **tools** attached at runtime (e.g. search, ask_human)
- A shared **logbook** used to record thoughts, actions, tool calls, and observations

In the next cell we create the global logbook,  load the configs, initialize the tools, and create the agents. 

In [ ]:
# Create the staff logbook
logbook = Logbook()

# Create legal agent
legal_config = yaml.safe_load(Path("agents/config/LegalAdvisor.yaml").read_text(encoding="utf-8"))

search = kb_search("agents/kb/LegalAdvisor", k=3)

legal = Agent(
    config = legal_config,
    logbook = logbook, 
    tools =  [ask_human, search],
)

# Create cyber operations agent
cyber_operations_config = yaml.safe_load(Path("agents/config/CyberOperationsExpert.yaml").read_text(encoding="utf-8"))

cyber_operations_search = kb_search("agents/kb/CyberOperationsExpert", k=3)

cyber_operations = Agent(
    config = cyber_operations_config,
    logbook = logbook,
    tools =  [ask_human, cyber_operations_search],
)

# Create military operations agent
military_operations_config = yaml.safe_load(Path("agents/config/MilitaryOperationsExpert.yaml").read_text(encoding="utf-8"))

military_operations_search = kb_search("agents/kb/MilitaryOperationsExpert", k=3)

military_operations = Agent(
    config = military_operations_config,
    logbook = logbook,
    tools =  [ask_human, military_operations_search],
)

# Create commander agent
commander_config = yaml.safe_load(Path("agents/config/Commander.yaml").read_text(encoding="utf-8"))

ask_legal = agent_as_tool(legal, "ask_legal")
ask_cyber_operations = agent_as_tool(cyber_operations, "ask_legal")
ask_military_operations = agent_as_tool(military_operations, "ask_legal")

commander = Agent(
    config = commander_config,
    logbook = logbook,
    tools =  [ask_human, ask_legal, ask_cyber_operations, ask_military_operations],
)

In [ ]:
# commander.reset()
# legal.reset()

## 2. Define the mission objective

The mission objective describes the operational problem to be analyzed. The objective is expressed as a multi-line natural-language description that
outlines the scenario and available Courses of Action (CoAs).  This text becomes  the first input to the Commander when the decision process begins.

In [ ]:
objective = """ 
Select the single best Course of Action (CoA) to prevent an imminent attack by the Terrmisous group on a commercial cargo ship carrying chemical agents near the civilian port AricikPortus.

Choose ONE option only (CoA 1, CoA 2, or CoA 3). Do not combine options.

CoA 1: Disable the VicikPortus power grid using previously implanted BlackEnergy3-based malware to prevent loading.
CoA 2: Temporarily neutralize the VicikPortus civilian pump station using a DDoS attack exploiting an unpatched vulnerability, preventing fueling.
CoA 3: Conduct GNSS spoofing to alter the cargo ship’s position, velocity, and heading after departure so it cannot reach its intended target.

"""

## 3. Run the decision cycle

With the staff, tools, and mission objective defined, we can now execute the full decision cycle.  

In [ ]:
commander.query(objective)

In [ ]:
# Text
text = logbook.console.export_text()

# HTML
html = logbook.console.export_html(inline_styles=True)

# Raw trace
trace = logbook.trace